In [16]:
import json
import pandas as pd

In [33]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re #regex for cleaning

ModuleNotFoundError: No module named 'spacy'

In [18]:
import fasttext

ModuleNotFoundError: No module named 'fasttext'

In [19]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [81]:
data = []
with open('./data/raw/r_republican_23posts.jsonl', 'r') as file:
    # Iterate through each line in the JSONL file
    for json_str in file:
        # Load each line as a JSON object
        result = json.loads(json_str)
        
        # Extract the desired fields if they exist and append them to the list
        data.append({
            'selftext': result.get('selftext', None),
            'title': result.get('title', None),
            'created': result.get('created', None),
            'author': result.get('author', None)
        })

# Create a pandas DataFrame from the list
df = pd.DataFrame(data)
df.head()

,selftext,title,created,author
0,,"Migrants Occupy $288 a Night NYC Hotel, Refuse...",NaN,yuri_2022
1,,BREAKING: Nikki Haley to run for president in ...,NaN,yuri_2022
2,,It's the land of the freebies for NYC's 'entit...,NaN,yuri_2022
3,,[VIDEO] Joe Biden Has Another Outburst and Sta...,NaN,SharkolyPamuk
4,[removed],Liberals giving this man 180 sentence only,NaN,ZoneZeus123


In [82]:
#concatenate text data so we have just an author/text per post

#remove "[removed]" from selftext field
df['selftext'] = df['selftext'].replace('[removed]', '', regex=False)

#create text column
df['text'] = df['selftext'].fillna('') + " " + df['title']
data = df[['author', 'text']]

In [83]:
#based off evolving divergence paper
#removes long posts that are duplicates in author/text
#removes repetitive URLS
def filter_spam(df):
    df = df[(df['text'].str.len() <= 90) | 
                    ((df['text'].str.len() > 90) & 
                     ~df.duplicated(subset=['author', 'text']))]
    
    #remove bot - like patterns?
    
    #extract spammy urls
    df.loc[:, 'url'] = df['text'].str.extract(r"(https://\S+)")
    spam_urls = df['url'].value_counts().nlargest(30).index

    df = df[~df['url'].isin(spam_urls)]  # remove posts containing spam urls
    
    
    #remove user/subreddit mentions
    df['text'] = df['text'].str.replace(r"(/r/\w+)", " ", regex=True)
    df['text'] = df['text'].str.replace(r"(/u/\w+)", " ", regex=True)
    
    #remove urls
    df['text'] = df['text'].str.replace(r"http[s]?://\S+", " ", regex=True)
    
    #extra spaces
    df['text'] = df['text'].str.replace(r"\s+", " ", regex=True)\
        .str.strip()          #leading/trailing spaces
    
    return df
        
 

In [84]:
#create new df with sentences, authors instead of post_text, authors
def to_sentences(row):
    author = row['author']
    text = row['text']
    
    all_sent = sent_tokenize(text)
    
    df_sent = [{'author': author, 'sentence': sentence} for sentence in all_sent]
    
    return df_sent

def text_cleaner(post):
    
    post = post.lower()  #lower case all text
    
    post = re.sub(r'[^a-zA-Z0-9\s]', '', post) #remove special char
    
    #will add more specific substitutions
    #removing numbers
    #removing abbrev/acronyms
    
    return post

In [85]:
#removes stop words, and lemmatizes
def lemma_er(post):
    
    stop_words = set(stopwords.words('english'))
    post = [word for word in post if word not in stop_words]
    
    post = [lemmatizer.lemmatize(word) for word in post]
    
    return post


In [86]:
#main
data = filter_spam(data)

#split text into sentences - nec for fasttext

sentence_data = df.apply(to_sentences, axis=1)
flat_sentences = [item for sublist in sentence_data for item in sublist]
sents = pd.DataFrame(flat_sentences)

sents.head()



/var/folders/z_/pjq4h8255nj3yc53_06_63440000gn/T/ipykernel_53423/1848685688.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'url'] = df['text'].str.extract(r"(https://\S+)")


,author,sentence
0,yuri_2022,"Migrants Occupy $288 a Night NYC Hotel, Refus..."
1,yuri_2022,BREAKING: Nikki Haley to run for president in...
2,yuri_2022,It's the land of the freebies for NYC's 'enti...
3,SharkolyPamuk,[VIDEO] Joe Biden Has Another Outburst and St...
4,ZoneZeus123,Liberals giving this man 180 sentence only


In [87]:
#clean text
sents['clean_text'] = sents['sentence'].apply(text_cleaner)

sents['tokens'] = sents['clean_text'].apply(word_tokenize)
#remove rows 


#THERE IS AN ISSUE WITH THE WORD_TOKENIZE


In [88]:
sents['tokens'] = sents['clean_text'].apply(lambda x: word_tokenize(x) if x.strip() else [])

#remove empty token rows
sents = sents[sents['tokens'].apply(lambda x: len(x) > 0)]


In [71]:
sents.iloc[2040:]

,author,sentence,clean_text,tokens
2075,agjrpsl,TINY D wants to break up GOOGLE.,tiny d wants to break up google,"[tiny, d, wants, to, break, up, google]"
2076,LengthinessTall166,Trump needs a lesson on jail lingo,trump needs a lesson on jail lingo,"[trump, needs, a, lesson, on, jail, lingo]"
2077,Excellent-Gold6560,"""But it's (D)ifferent"" nope hope you you",but its different nope hope you you,"[but, its, different, nope, hope, you, you]"
2078,futurific,Man arrested for alleged child molestation du...,man arrested for alleged child molestation du...,"[man, arrested, for, alleged, child, molestati..."
2079,CarRemarkable2704,And Another one Gone .... And Another one Gon...,and another one gone and another one gone,"[and, another, one, gone, and, another, one, g..."
...,...,...,...,...
12025,agjrpsl,Trump Co-Defendant in Georgia Election Interf...,trump codefendant in georgia election interfe...,"[trump, codefendant, in, georgia, election, in..."
12026,agjrpsl,Trump’s Promise of Lawlessness,trumps promise of lawlessness,"[trumps, promise, of, lawlessness]"
12027,RealityUSA2023,BREAKING: The FBI arrests the fugitive member...,breaking the fbi arrests the fugitive member ...,"[breaking, the, fbi, arrests, the, fugitive, m..."
12028,wdcmsnbcgay,Homophobe Matt Gaetz Seeks Democratic Allianc...,homophobe matt gaetz seeks democratic allianc...,"[homophobe, matt, gaetz, seeks, democratic, al..."


In [89]:


lemmatizer = WordNetLemmatizer()
sents.loc[:, 'out_sent'] = sents['tokens'].apply(lemma_er)
sents.head()

,author,sentence,clean_text,tokens,out_sent
0,yuri_2022,"Migrants Occupy $288 a Night NYC Hotel, Refus...",migrants occupy 288 a night nyc hotel refuse ...,"[migrants, occupy, 288, a, night, nyc, hotel, ...","[migrant, occupy, 288, night, nyc, hotel, refu..."
1,yuri_2022,BREAKING: Nikki Haley to run for president in...,breaking nikki haley to run for president in ...,"[breaking, nikki, haley, to, run, for, preside...","[breaking, nikki, haley, run, president, 2024]"
2,yuri_2022,It's the land of the freebies for NYC's 'enti...,its the land of the freebies for nycs entitle...,"[its, the, land, of, the, freebies, for, nycs,...","[land, freebie, nycs, entitled, migrant]"
3,SharkolyPamuk,[VIDEO] Joe Biden Has Another Outburst and St...,video joe biden has another outburst and star...,"[video, joe, biden, has, another, outburst, an...","[video, joe, biden, another, outburst, start, ..."
4,ZoneZeus123,Liberals giving this man 180 sentence only,liberals giving this man 180 sentence only,"[liberals, giving, this, man, 180, sentence, o...","[liberal, giving, man, 180, sentence]"


In [90]:
sents = sents[['author', 'out_sent']]

sents.to_csv('./data/utf8-lemma/reps_sents_p23.csv', encoding='utf-8')

In [91]:
sents['out_sent'].iloc[5]

['troubled', 'relation', 'defining', 'successor', 'ideology']